In [1]:
import pandas as pd
import fiddler as fdl


In [ ]:
URL = 'https://customer.fiddler.ai'
TOKEN = ''  # Get this from the Settings > Credentials tab in Fiddler UI


fdl.init(url=URL, token=TOKEN)
print(f"Successfully connected to Fiddler at {URL}")
print(f"Client version:    {fdl.__version__}")
print(f"Server version:    {fdl.conn.server_version}")
print(f"Organization ID:   {fdl.conn.organization_id}")
print(f"Organization name: {fdl.conn.organization_name}")

In [ ]:
FEATURE_IMPORTANCE_CSV = 'temp_featimp/app_engine_feature_imp.csv'
df = pd.read_csv(FEATURE_IMPORTANCE_CSV)
feature_importance_dict = df.set_index('feature_name')['feature_imp'].to_dict()

df.head()

In [ ]:
PROJECT_NAME = 'upsell_risk_prod'  # Project name
MODEL_NAME = 'appen_upsell_model_prod'  # Model name
VERSION = 'v6'


project = fdl.Project.from_name(name=PROJECT_NAME)
project_id = project.id
print(f"Using project: {project.name} (ID: {project_id})")

model = fdl.Model.from_name(name=MODEL_NAME, project_id=project_id, version=VERSION)
print(f"Retrieved model: {model.name}, version: {model.version} (ID: {model.id})")


In [ ]:
model_inputs = model.spec.inputs
missing_features = set(model_inputs) - set(feature_importance_dict.keys())
extra_features = set(feature_importance_dict.keys()) - set(model_inputs)

if missing_features:
    print(f"Warning: The following model input features are missing from the CSV: {missing_features}")

if extra_features:
    print(f"Warning: The following features in the CSV are not model inputs: {extra_features}")
    # Remove extra features to prevent upload errors
    for feature in extra_features:
        del feature_importance_dict[feature]

# Upload the feature importance values
result = model.upload_feature_impact(feature_impact_map=feature_importance_dict, update=True)
print(f"Successfully uploaded feature importance values to model {model.name}")
